In [1]:
import spacy
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import pandas as pd
spacy.cli.download("en_core_web_md")

✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')


In [2]:
df = pd.read_parquet('../Datasets_ML/Rest_google.parquet')
df.head()

,name,address,description,category,MISC,coord
0,Roux's Roadhouse 73,"Roux's Roadhouse 73, 784 WI-73, Nekoosa, WI 54457",None,[Restaurant],{'Accessibility': ['Wheelchair accessible entr...,"[44.3069541, -89.8457834]"
1,Crepes n' Tacos Mexican Grill,"Crepes n' Tacos Mexican Grill, suit #5, 5390, ...",None,"[Mexican restaurant, Crêperie, Ice cream shop,...","{'Accessibility': None, 'Amenities': ['Good fo...","[36.1646332, -115.0607804]"
2,The Big Eazy,"The Big Eazy, 2053 N Doctor M.L.K. Jr Dr, Milw...",None,"[Cajun restaurant, Creole restaurant]","{'Accessibility': None, 'Amenities': ['Good fo...","[43.0576581, -87.914643]"
3,Subway,"Subway, 250 E Wolf Run, Mukwonago, WI 53149",Casual counter-serve chain for build-your-own ...,"[Sandwich shop, Caterer, Fast food restaurant,...","{'Accessibility': None, 'Amenities': None, 'At...","[42.8541679, -88.3142471]"
4,Papa Kelsey's,"Papa Kelsey's, 165 E 1400 N, Logan, UT 84341","Chain eatery serving baked subs, classic pizza...",[Pizza restaurant],{'Accessibility': ['Wheelchair accessible entr...,"[41.7580941, -111.8302904]"


In [3]:
nlp_en = spacy.load("en_core_web_md")

In [9]:
def buscar_palabras_similares_avanzado_ingles(dataframe, columna, lista_busqueda, umbral_similitud=0.7):
    indices = []

    for index,lista in enumerate(dataframe[columna]):
        for palabra in lista:
            similitudes = []
            for palabra_busqueda in lista_busqueda:
                # Calcular la similitud coseno entre los embeddings de las palabras
                embedding_palabra = nlp_en(palabra).vector.reshape(1, -1)
                embedding_palabra_busqueda = nlp_en(palabra_busqueda).vector.reshape(1, -1)
                similitud = cosine_similarity(embedding_palabra, embedding_palabra_busqueda)[0][0]
                similitudes.append(similitud)
            
            # Si la similitud es mayor que el umbral, agregar la palabra
            if max(similitudes) >= umbral_similitud:
                indices.append(index)
                break
    return indices

In [10]:
# Lista de palabras para buscar
palabras_busqueda = ['sushi', 'beer','delivery']

# Buscar palabras iguales o similares en la columna 'palabras'
palabras_similares = buscar_palabras_similares_avanzado_ingles(df, 'category', palabras_busqueda)

print("indices", list(set(palabras_similares)))

Palabras iguales o similares encontradas: [513, 1540, 2053, 2055, 1545, 14, 3087, 3089, 1554, 3090, 1556, 22, 23, 1562, 1051, 3098, 2589, 1057, 2593, 2551, 1575, 2599, 3111, 3115, 46, 3120, 3121, 3063, 53, 568, 59, 2620, 1085, 1597, 63, 2621, 578, 3138, 70, 3142, 1101, 3153, 1621, 608, 2146, 3173, 2151, 2663, 1645, 3184, 115, 629, 3191, 2680, 636, 1149, 2172, 2175, 2689, 1155, 3207, 136, 2701, 658, 2194, 148, 150, 671, 160, 2208, 674, 1699, 1704, 170, 1196, 2735, 176, 688, 2743, 2745, 189, 2237, 703, 2753, 2246, 2760, 713, 2761, 204, 206, 2255, 1234, 2771, 1236, 1750, 732, 2781, 734, 735, 2782, 2785, 2788, 743, 744, 746, 2282, 1772, 2284, 2796, 1776, 2289, 2801, 755, 757, 2808, 249, 251, 2813, 2304, 769, 2816, 773, 2823, 776, 2824, 276, 2838, 1303, 2332, 797, 1822, 2337, 2338, 291, 2849, 2350, 1840, 1841, 2352, 307, 820, 1331, 1846, 2874, 317, 831, 320, 839, 840, 846, 1872, 2896, 1362, 2905, 348, 1886, 1888, 2401, 1893, 1384, 2410, 2923, 876, 368, 1907, 2933, 1913, 381, 391, 2441, 1418